# APPLICATIONS OF MARKOV DECISION PROCESSES
---
In this notebook we will take a look at some indicative applications of markov decision processes. 
We will cover content from [`mdp.py`](https://github.com/aimacode/aima-python/blob/master/mdp.py), for chapter 17 of Stuart Russel's and Peter Norvig's book [*Artificial Intellignece: A Modern Approach*](http://aima.cs.berkeley.edu/).

In [1]:
from mdp import *
from notebook import psource, pseudocode

## CONTENTS
- Simple MDPs


## SIMPLE MDP

Markov Decision Processes are formally described as processes that follow the Markov property which states that "The future is independent of the past given the present". 
MDPs formally describe environments for reinforcement learning and we assume that the environment is *fully observable*. 
Let us take a toy example MDP and solve it using the functions in `mdp.py`.
This is a simple example adapted from a [similar problem](http://www0.cs.ucl.ac.uk/staff/D.Silver/web/Teaching_files/MDP.pdf) by Dr. David Silver, tweaked to fit the limitations of the current functions.
![title](images/mdp-b.png)

Let's say you're a student attending lectures in a university.
There are three lectures you need to attend on a given day.
<br>
Attending the first lecture gives you 4 points of reward.
After the first lecture, you have a 0.6 probability to continue into the second one, yielding 6 more points of reward.
But, with a probability of 0.4, you get distracted and start using Facebook instead and get a reward of -1.
From then onwards, you really can't let go of Facebook and there's just a 0.1 probability that you will concentrate back on the lecture.
<br>
After the second lecture, you have an equal chance of attending the next lecture or just falling asleep.
Falling asleep is the terminal state and yields you no reward, but continuing on to the final lecture gives you a big reward of 10 points.
<br>
From there on, you have a 40% chance of going to study and reach the terminal state, 
but a 60% chance of going to the pub with your friends instead. 
You end up drunk and don't know which lecture to attend, so you go to one of the lectures according to the probabilities given above.
<br> 
We now have an outline of our stochastic environment and we need to maximize our reward by solving this MDP.
<br>
<br>
We first have to define our Transition Matrix as a nested dictionary to fit the requirements of the MDP class.

In [2]:
t = {
    'leisure': {
                    'facebook': {'leisure':0.9, 'class1':0.1},
                    'quit': {'leisure':0.1, 'class1':0.9},
                    'study': {},
                    'sleep': {},
                    'pub': {}
               },
    'class1': {
                    'study': {'class2':0.6, 'leisure':0.4},
                    'facebook': {'class2':0.4, 'leisure':0.6},
                    'quit': {},
                    'sleep': {},
                    'pub': {}
              },
    'class2': {
                    'study': {'class3':0.5, 'end':0.5},
                    'sleep': {'end':0.5, 'class3':0.5},
                    'facebook': {},
                    'quit': {},
                    'pub': {},
              },
    'class3': {
                    'study': {'end':0.6, 'class1':0.08, 'class2':0.16, 'class3':0.16},
                    'pub': {'end':0.4, 'class1':0.12, 'class2':0.24, 'class3':0.24},
                    'facebook': {},
                    'quit': {},
                    'sleep': {}
              },
    'end': {}
}

We now need to define the reward for each state.

In [3]:
rewards = {
    'class1': 4,
    'class2': 6,
    'class3': 10,
    'leisure': -1,
    'end': 0
}

This MDP has only one terminal state.

In [4]:
terminals = ['end']

Let's now set the initial state to Class 1.

In [5]:
init = 'class1'

We will write a CustomMDP class to extend the MDP class for the problem at hand. 
This class will implement the `T` method to implement the transition model. This is the exact same class as given in [`mdp.ipynb`](https://github.com/aimacode/aima-python/blob/master/mdp.ipynb#MDP).

In [6]:
class CustomMDP(MDP):

    def __init__(self, transition_matrix, rewards, terminals, init, gamma=.9):
        # All possible actions.
        actlist = []
        for state in transition_matrix.keys():
            actlist.extend(transition_matrix[state])
        actlist = list(set(actlist))
        print(actlist)

        MDP.__init__(self, init, actlist, terminals=terminals, gamma=gamma)
        self.t = transition_matrix
        self.reward = rewards
        for state in self.t:
            self.states.add(state)

    def T(self, state, action):
        if action is None:
            return [(0.0, state)]
        else: 
            return [(prob, new_state) for new_state, prob in self.t[state][action].items()]

We now need an instance of this class.

In [7]:
mdp = CustomMDP(t, rewards, terminals, init, gamma=.9)

['sleep', 'study', 'pub', 'facebook', 'quit']


The utility of each state can be found by `value_iteration`.

In [8]:
value_iteration(mdp)

{'class1': 16.90340650279542,
 'class2': 14.597383430869879,
 'class3': 19.10533144728953,
 'end': 0.0,
 'leisure': 13.946891353066082}

Now that we can compute the utility values, we can find the best policy.

In [9]:
pi = best_policy(mdp, value_iteration(mdp, .01))

`pi` stores the best action for each state.

In [10]:
print(pi)

{'class3': 'pub', 'end': None, 'leisure': 'quit', 'class2': 'sleep', 'class1': 'study'}


We can confirm that this is the best policy by verifying this result against `policy_iteration`.

In [11]:
policy_iteration(mdp)

{'class1': 'study',
 'class2': 'sleep',
 'class3': 'pub',
 'end': None,
 'leisure': 'quit'}

Everything looks perfect, but let us look at another possibility for an MDP.
<br>
Till now we have only dealt with rewards that the agent gets while it is **on** a particular state.
What if we want to have different rewards for a state depending on the action that the agent takes next. 
The agent gets the reward _during its transition_ to the next state.
<br>
For the sake of clarity, we will call this the _transition reward_ and we will call this kind of MDP a _dynamic_ MDP. 
This is not a conventional term, we just use it to minimize confusion between the two.
<br>
This next section deals with how to create and solve a dynamic MDP.

Let us consider a very similar problem, but this time, we do not have rewards _on_ states, 
instead, we have rewards on the transitions between states. 
This state diagram will make it clearer.
![title](images/mdp-c.png)

A very similar scenario as the previous problem, but we have different rewards for the same state depending on the action taken.
<br>
To deal with this, we just need to change the `R` method of the MDP class, but to prevent confusion, we will write a new similar class `DMDP`.

In [12]:
class DMDP:

    """A Markov Decision Process, defined by an initial state, transition model,
    and reward model. We also keep track of a gamma value, for use by
    algorithms. The transition model is represented somewhat differently from
    the text. Instead of P(s' | s, a) being a probability number for each
    state/state/action triplet, we instead have T(s, a) return a
    list of (p, s') pairs. The reward function is very similar.
    We also keep track of the possible states,
    terminal states, and actions for each state. [page 646]"""

    def __init__(self, init, actlist, terminals, transitions={}, rewards={}, states=None, gamma=.9):
        if not (0 < gamma <= 1):
            raise ValueError("An MDP must have 0 < gamma <= 1")

        if states:
            self.states = states
        else:
            self.states = set()
        self.init = init
        self.actlist = actlist
        self.terminals = terminals
        self.transitions = transitions
        self.rewards = rewards
        self.gamma = gamma

    def R(self, state, action):
        """Return a numeric reward for this state."""
        if (self.rewards == {}):
            raise ValueError('Reward model is missing')
        else:
            return self.rewards[state][action]

    def T(self, state, action):
        """Transition model. From a state and an action, return a list
        of (probability, result-state) pairs."""
        if(self.transitions == {}):
            raise ValueError("Transition model is missing")
        else:
            return self.transitions[state][action]

    def actions(self, state):
        """Set of actions that can be performed in this state. By default, a
        fixed list of actions, except for terminal states. Override this
        method if you need to specialize by state."""
        if state in self.terminals:
            return [None]
        else:
            return self.actlist

The transition model will be the same

In [13]:
t = {
    'leisure': {
                    'facebook': {'leisure':0.9, 'class1':0.1},
                    'quit': {'leisure':0.1, 'class1':0.9},
                    'study': {},
                    'sleep': {},
                    'pub': {}
               },
    'class1': {
                    'study': {'class2':0.6, 'leisure':0.4},
                    'facebook': {'class2':0.4, 'leisure':0.6},
                    'quit': {},
                    'sleep': {},
                    'pub': {}
              },
    'class2': {
                    'study': {'class3':0.5, 'end':0.5},
                    'sleep': {'end':0.5, 'class3':0.5},
                    'facebook': {},
                    'quit': {},
                    'pub': {},
              },
    'class3': {
                    'study': {'end':0.6, 'class1':0.08, 'class2':0.16, 'class3':0.16},
                    'pub': {'end':0.4, 'class1':0.12, 'class2':0.24, 'class3':0.24},
                    'facebook': {},
                    'quit': {},
                    'sleep': {}
              },
    'end': {}
}

The reward model will be a dictionary very similar to the transition dictionary with a reward for every action for every state.

In [14]:
r = {
    'leisure': {
        'facebook':-1,
        'quit':0,
        'study':0,
        'sleep':0,
        'pub':0
    },
    'class1': {
        'study':-2,
        'facebook':-1,
        'quit':0,
        'sleep':0,
        'pub':0
    },
    'class2': {
        'study':-2,
        'sleep':0,
        'facebook':0,
        'quit':0,
        'pub':0
    },
    'class3': {
        'study':10,
        'pub':1,
        'facebook':0,
        'quit':0,
        'sleep':0
    },
    'end': {
        'study':0,
        'pub':0,
        'facebook':0,
        'quit':0,
        'sleep':0
    }
}

The MDP has only one terminal state

In [15]:
terminals = ['end']

Let's now set the initial state to Class 1.

In [16]:
init = 'class1'

We will write a CustomDMDP class to extend the DMDP class for the problem at hand.
This class will implement everything that the previous CustomMDP class implements along with a new reward model.

In [17]:
class CustomDMDP(DMDP):
    
    def __init__(self, transition_matrix, rewards, terminals, init, gamma=.9):
        actlist = []
        for state in transition_matrix.keys():
            actlist.extend(transition_matrix[state])
        actlist = list(set(actlist))
        print(actlist)
        
        DMDP.__init__(self, init, actlist, terminals=terminals, gamma=gamma)
        self.t = transition_matrix
        self.rewards = rewards
        for state in self.t:
            self.states.add(state)
            
            
    def T(self, state, action):
        if action is None:
            return [(0.0, state)]
        else:
            return [(prob, new_state) for new_state, prob in self.t[state][action].items()]
        
    def R(self, state, action):
        if action is None:
            return 0
        else:
            return self.rewards[state][action]

One thing we haven't though about yet is that the `value_iteration` algorithm won't work now that the reward model is changed.
It will be quite similar to the one we currently have nonetheless.

The Bellman update equation now is defined as follows

$$U(s)=\max_{a\epsilon A(s)}\sum_{s'}P(s'\ |\ s,a)(R(s'\ |\ s,a) + \gamma U(s'))$$